In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers, losses
from keras.models import Model
import udata
import os

In [2]:
# train data
x_train = udata.outdata()
x_test = x_train
y_data = udata.indata()

print(np.shape(x_train))

(65, 1325705, 3)


In [4]:
latent_dim = 8
input_size = len(x_train[0])
layer_size = input_size * 3

In [5]:
class Autoencoder(Model):
  def __init__(self, encoding_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
        layers.Flatten(),
        layers.Dense(latent_dim, activation='tanh'),
    ])
    self.decoder = tf.keras.Sequential([
        layers.Dense(layer_size, activation='linear'),
        layers.Reshape((input_size,3))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)

autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [6]:
autoencoder.fit(x_train, x_train, batch_size=13,
                epochs=2000,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/2000
5/5 [==============================] - 10s 2s/step - loss: 3.6548 - accuracy: 0.4020 - val_loss: 3.6486 - val_accuracy: 0.5429
Epoch 2/2000
5/5 [==============================] - 4s 751ms/step - loss: 3.6428 - accuracy: 0.5723 - val_loss: 3.6325 - val_accuracy: 0.5944
Epoch 3/2000
5/5 [==============================] - 4s 835ms/step - loss: 3.6254 - accuracy: 0.5982 - val_loss: 3.6131 - val_accuracy: 0.6009
Epoch 4/2000
5/5 [==============================] - 4s 770ms/step - loss: 3.6048 - accuracy: 0.5986 - val_loss: 3.5920 - val_accuracy: 0.6041
Epoch 5/2000
5/5 [==============================] - 4s 989ms/step - loss: 3.5827 - accuracy: 0.6071 - val_loss: 3.5698 - val_accuracy: 0.6079
Epoch 6/2000
5/5 [==============================] - 4s 933ms/step - loss: 3.5609 - accuracy: 0.6077 - val_loss: 3.5465 - val_accuracy: 0.6096
Epoch 7/2000
5/5 [==============================] - 5s 1s/step - loss: 3.5369 - accuracy: 0.6095 - val_loss: 3.5227 - val_accuracy: 0.6099
Epoch 8/200

In [7]:
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (13, 8)                   31816928  
                                                                 
 sequential_1 (Sequential)   (13, 1325705, 3)          35794035  
                                                                 
Total params: 67,610,963
Trainable params: 67,610,963
Non-trainable params: 0
_________________________________________________________________


In [9]:
encoded_u = autoencoder.encoder(x_test).numpy()
decoded_u = autoencoder.decoder(encoded_u).numpy()

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [8]:
os.chdir('../')
os.chdir('./result')

# cnn 학습을 위해 저장
print(np.shape(encoded_u))
getdata=np.hstack((y_data,encoded_u))
np.savetxt('invisible2000.txt',getdata,delimiter=",")
# print(getdata)  # 풍속과 은닉층 값 합친것


(65, 8)


In [9]:
# 디코더 성능을 알아보기 위해 디코더로 생성된 U' 값중 하나 저장
print(np.shape(decoded_u))
test1cfd = decoded_u[0,:,:]
# np.set_printoptions(formatter={'float_kind': lambda x:"{0:0.5f}".format(x)})
test1cfd = np.round_(test1cfd,5)
# print(test1cfd.dtype)
print(test1cfd)
np.savetxt('test1.txt', test1cfd, delimiter=" ")
# test2cfd = decoded_u([-0.99999803 -0.99877566  0.96006227 -0.9995706  -1.          0.9999979
#    0.96989375 -0.97782224])

(65, 1325705, 3)
[[-1.02500e-02  2.20000e-04  4.72400e-02]
 [-2.61355e+00 -2.61816e+00 -4.68370e-01]
 [-1.23535e+00 -2.40010e-01  2.64200e-02]
 ...
 [-4.91490e+00 -3.27127e+00  4.58970e-01]
 [-4.99627e+00 -3.31114e+00  4.39250e-01]
 [-5.05957e+00 -3.33836e+00  4.18360e-01]]


In [10]:
# cnn으로 생성한 히든 레이어 값으로 디코딩
prediction = np.array([[ 0.97716624, -0.9967411,  -0.990131,   -0.99553293,  0.24914098 , 0.972963 ,  0.9219302,  -0.9773568 ]])
pred_u = autoencoder.decoder(prediction).numpy()
print(np.shape(pred_u))

(1, 1325705, 3)


In [11]:
# CNN으로 생성한 값으로 디코딩 한 값을 저장
cnnpred1 = pred_u[0,:,:]
# np.set_printoptions(formatter={'float_kind': lambda x:"{0:0.5f}".format(x)})
cnnpred1 = np.round_(cnnpred1,5)
print(cnnpred1)
np.savetxt('cnnpred.txt', cnnpred1, delimiter=" ")

[[-2.04888  0.98117  0.26333]
 [-2.61493  1.74734  0.51205]
 [-3.29506  1.09499  0.58055]
 ...
 [-4.19281  2.64183  0.27448]
 [-4.2204   2.64537  0.2783 ]
 [-4.23264  2.6516   0.28023]]
